<a href="https://colab.research.google.com/github/LorenzoLeon/bourbaki_NB_analisis_texto/blob/main/02_script/Exploracion_BD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Los paquetes a usar. Vienen con bibliotecas simples para descargar objetos y bases de datos útiles

In [1]:
!pip3 install pickle5
import pickle5 as pickle
import pandas as pd
# Visualización
import matplotlib.pyplot as plt
import matplotlib
from wordcloud import WordCloud

#Aquí copiamos las bases de datos a nuestro directorio en la nube
! git clone https://github.com/LorenzoLeon/bourbaki_NB_analisis_texto.git

     |████████████████████████████████| 133kB 12.3MB/s 
  Created wheel for pickle5: filename=pickle5-0.0.11-cp37-cp37m-linux_x86_64.whl size=219248 sha256=5995b8ab043d3523f16f06a2518e982e9d5eb75aed9ed0f7270752d6089a6e76
  Stored in directory: /root/.cache/pip/wheels/a6/90/95/f889ca4aa8b0e0c7f21c8470b6f5d6032f0390a3a141a9a3bd
Successfully built pickle5
Cloning into 'bourbaki_NB_analisis_texto'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 20 (delta 4), reused 14 (delta 1), pack-reused 0
Unpacking objects: 100% (20/20), done.


Primero, abramos una base de datos. Hoy vamos a utilizar una lista de tweets mexicanos que usaron el hashtag #covid (o derivados). Estos tweets fueron recopilados directamente desde Twitter.
Por ahora, este archivo es un pickle, o un objeto de python que alguien ya formó (yo) y que guardó para su fácil carga. 
Ustedes solo tendrán que leer el archivo, e inmediatamente tendrán un objeto formato base de datos de Pandas.

In [137]:
tweets_covid = pd.read_pickle('bourbaki_NB_analisis_texto/01_datos/tweets_small.pickle')

¿Qué tiene esta base de datos?
Cómo es pandas, podemos acceder a las columnas o filas de la base de datos fácilmente.

In [138]:
len(tweets_covid)

22000

In [139]:
tweets_covid.dtypes

id_str              int64
text               object
mentions           object
hashes             object
user_name          object
retweeted           int64
inreply           float64
user_followers      int64
fecha              object
type               object
value               int64
value2              int64
dtype: object

In [140]:
tweets_covid["fecha"][139]

'2020-11-26T00:32:20Z'

In [141]:
tweets_covid["fecha"]= pd.DatetimeIndex(tweets_covid['fecha']).tz_convert("America/Mexico_City")
tweets_covid["fecha"][139]

Timestamp('2020-11-25 18:32:20-0600', tz='America/Mexico_City')

In [142]:
tweets_covid.fecha.describe(datetime_is_numeric=True)

count                                  21999
mean     2020-11-19 23:37:22.465826304-06:00
min                2020-07-16 02:16:51-05:00
25%                2020-08-27 09:29:19-05:00
50%                2020-12-01 07:46:21-06:00
75%                2021-01-25 22:34:50-06:00
max                2021-04-05 15:51:43-05:00
Name: fecha, dtype: object

In [143]:
tweets_covid['type'] = tweets_covid['type'].astype('category')
tweets_covid.value2.describe()

count    22000.000000
mean         0.000000
std          1.000023
min         -1.000000
25%         -1.000000
50%          0.000000
75%          1.000000
max          1.000000
Name: value2, dtype: float64

Aquí podemos ver fácilmente las primeras 5 entradas en nuestra base de datos.

In [144]:
tweets_covid.sort_index().head()

,id_str,text,mentions,hashes,user_name,retweeted,inreply,user_followers,fecha,type,value,value2
69,1331766676082155520,@h43226013 @Claudiashein Y cuántos niños mexic...,"@h43226013, @Claudiashein","#GobiernoCriminalyCorrupto, #ElPeorGobiernoDeL...",h43226013,1,NaN,1351,2020-11-25 18:08:13-06:00,negative,99,-1
139,1331772748134281220,"? #ÚLTIMAHORA | Por SEGUNDO día consecutivo, s...","@lopezobrador_, @HLGatell","#ÚLTIMAHORA, #Covid_19, #México",MaryTeTe31,1,NaN,351,2020-11-25 18:32:20-06:00,positive,88,1
218,1331780180260622336,Reconoce el Subsecretario de Salud @HLGatell l...,@HLGatell,NaN,Micheguti,1,NaN,90,2020-11-25 19:01:52-06:00,positive,60,1
307,1331788520436936704,@Claudiashein Como jodes con Maradona. Y los ...,@Claudiashein,NaN,Albertus2021,0,1.331724e+18,516,2020-11-25 19:35:01-06:00,negative,100,-1
319,1331790707451965441,Reconoce el Subsecretario de Salud @HLGatell l...,@HLGatell,NaN,IDEY_oficial,1,NaN,4996,2020-11-25 19:43:42-06:00,positive,60,1


In [145]:
tweets_covid.hashes = tweets_covid.hashes.astype(str)

hashes = tweets_covid.hashes[tweets_covid.hashes != "nan"]

hashes.str.split(", ",expand=True).stack().value_counts().head(30)

#COVID19                    4610
#Covid_19                    888
#Covid_19mx                  829
#Covid19                     807
#COVIDー19                    471
#COVID19mx                   405
#COVID                       355
#CuidemosTodosDeTodos        286
#QuedateEnCasa               283
#coronavirus                 278
#COVID__19                   273
#México                      269
#VacunaCOVID19               266
#QuédateEnCasa               251
#Covid                       248
#CovidDerrotaAPuebla         243
#UsaCubrebocas               228
#Oaxaca                      212
#CDMX                        197
#Edoméx                      181
#SputnikV                    166
#Reto40xOax                  157
#DoctorMuerte                147
#SanaDistancia               142
#PorAmorALaVida              138
#MeCuido                     137
#vacuna                      129
#SuSanaDistancia             126
#Covidiotas                  123
#EdoméxEnSemáforoNaranja     122
dtype: int

In [146]:
for i in tweets_covid.index[:10]:
  print("texto: "  + str(tweets_covid["text"][i]))
  print("usuario: " + tweets_covid["user_name"][i])
  print("sentimiento:"+ tweets_covid["type"][i])
  print("######")

texto: ?#EnVivo | Ya inicia #CruceInformativo con @YonAmador ?:

??La Jefa de Gobierno, @Claudiashein, da positivo a #COVID19

??@lopezobrador_ declaró tres días de luto nacional en memoria de fallecidos por la pandemia 

#DerechoALaInformación

https://t.co/5ExdY42xus
usuario: C21Noticias
sentimiento:negative
######
texto: Por instrucción presidencial antes de volver al rojo, la @Claudiashein tiene primero que explotar todos los anaranjados posibles o llegar a 120 mil muertes por #COVIDー19 lo que ocurra primero!!

#MorenaLaPandemiaDeMexico https://t.co/urPCNQAIpU
usuario: MarioPe62068812
sentimiento:negative
######
texto: @omarfayad El gobernador @OmarFayad primer donador de plasma contra el COVID-19 en #Hidalgo ,..
usuario: TeresaC31916268
sentimiento:negative
######
texto: Este video debería ser retuiteado miles de veces hasta que @HLGatell tenga pesadillas con él. Ojalá que México no olvide NUNCA lo que este @GobiernoMX de tres centavos y este individuo hizo con la crisis del COVID